In [1]:
import numpy as np
import pandas as pd

from copy import deepcopy
from functools import partial
import matplotlib.pyplot as plt
import optuna
import pickle
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [2]:
# leaked_df = pd.read_csv('../input/leaked_data_all.csv', parse_dates=['timestamp'])
with open('../input/leak_data_drop_bad_rows.pkl', 'rb') as f:
    leaked_df = pickle.load(f).rename(columns={'meter_reading': 'leaked_meter_reading'})

# leaked_df = pd.read_feather('../input/leak_data.feather').rename(columns={'meter_reading': 'leaked_meter_reading'})
leaked_df = leaked_df[['building_id','meter','timestamp', 'leaked_meter_reading']]
leaked_df = leaked_df.query('timestamp>=20170101')

building_meta = pd.read_csv("../input/building_metadata.csv")

leaked_df = leaked_df.merge(building_meta[['building_id', 'site_id']], on='building_id', how='left')

leaked_df = leaked_df.query('~(meter==0 & site_id==0)')
# leaked_df = leaked_df.query('site_id==[2,4,15]')
# leaked_df = leaked_df.query('105<=building_id<=564 | 656<=building_id')

test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])

In [3]:
i = 1

for leave in [7,15,31, 63]:
    for lr in ["001","005","01"]:
        submission_s1 = pd.read_csv(f'../output/use_train_fe_seed1_leave{leave}_lr{lr}_mul05.csv')
#         submission_s2 = pd.read_csv(f'../output/use_train_fe_seed2_leave{leave}_lr{lr}_mul05.csv')
#         submission_s3 = pd.read_csv(f'../output/use_train_fe_seed3_leave{leave}_lr{lr}_mul05.csv')
#         test[f'pred{i}'] = (submission_s1['meter_reading'] + submission_s2['meter_reading'] + submission_s3['meter_reading']) / 3
        test[f'pred{i}'] = submission_s1['meter_reading']
        i += 1
# del submission_s1, submission_s2, submission_s3

FileNotFoundError: [Errno 2] File b'../output/use_train_fe_seed1_leave31_lr001_mul05.csv' does not exist: b'../output/use_train_fe_seed1_leave31_lr001_mul05.csv'

In [4]:
# submission11 = pd.read_csv('../output/fe2_lgbm.csv')
# submission12 = pd.read_csv('../output/submission_tomioka.csv')
# submission13 = pd.read_csv('../output/submission_half_and_half.csv')
# submission14 = pd.read_csv('../output/submission_distill.csv')

In [5]:
# test['pred11'] = np.exp(1) - 1

In [6]:
test = test.merge(leaked_df, on=['building_id', 'meter', 'timestamp'], how='left')
N = test.columns.str.startswith('pred').sum()
print(N)

6


In [7]:
test_sub = test.copy()
test = test[~test['leaked_meter_reading'].isnull()]
test2017 = test.query('timestamp<20180101')
test2018 = test.query('20180101<=timestamp')

In [8]:
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(test2018[f'pred{i}']))))

print()

for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(test2017[f'pred{i}']))))

print()
    
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(test[f'pred{i}']))))


1 0.8865035729434979
2 0.8027040608088559
3 0.7986760807250235
4 0.8591574841491776
5 0.7981638094671428
6 0.7967608101420031

1 0.8952273203538572
2 0.7829052029232684
3 0.7741473995216276
4 0.855230482461891
5 0.7707523357204036
6 0.7661268767700412

1 0.8909050023878055
2 0.7928008966593048
3 0.7864261757027987
4 0.8571832324742487
5 0.7844870622737529
6 0.781492547072532


In [9]:
# sid = 1
# for i in range(1,N+1):
#     print(i, np.sqrt(mean_squared_error(np.log1p(test2018.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
#                                         np.log1p(test2018.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))

# print()

# for i in range(1,N+1):
#     print(i, np.sqrt(mean_squared_error(np.log1p(test2017.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
#                                         np.log1p(test2017.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))
       

In [10]:
focus_df = test2018

In [11]:
def preproceeding(submission, N):
    submission.loc[:,'pred1':'leaked_meter_reading'] = np.log1p(submission.loc[:,'pred1':'leaked_meter_reading'])
    g = submission.groupby('meter')
    sub_sub = [dict(), dict(), dict(), dict()]
    leak_sub = [dict(), dict(), dict(), dict()]
    leak_leak = [0,0,0,0]
    
    for meter in [3,2,1,0]:
        for i in tqdm(range(1,N+1)):
            leak_sub[meter][i] = sum(-2 * g.get_group(meter)['leaked_meter_reading'] * g.get_group(meter)[f'pred{i}'])
            for j in range(1,N+1):
                if i > j: 
                    sub_sub[meter][(i,j)] = sub_sub[meter][(j,i)]
                else:
                    sub_sub[meter][(i,j)] = sum(g.get_group(meter)[f'pred{i}'] * g.get_group(meter)[f'pred{j}'])
        
        leak_leak[meter] = (sum(g.get_group(meter)['leaked_meter_reading'] ** 2))
    
    return sub_sub, leak_sub, leak_leak

In [12]:
sub_sub, leak_sub, leak_leak = preproceeding(focus_df.copy(), N)

100%|██████████| 6/6 [00:13<00:00,  2.19s/it]


In [13]:
count_itr = 0
def optimization(meter, sub_sub, leak_sub, leak_leak, length, W):
    
#     global count_itr
#     if count_itr%1000 == 0: print(count_itr, end=' ')
#     count_itr += 1
    
    loss_total = 0

    for i, a in enumerate(W, 1):
        for j, b in enumerate(W, 1):
            loss_total += a * b * sub_sub[meter][(i, j)]

    for i, a in enumerate(W, 1):
        loss_total += leak_sub[meter][i] * a

    loss_total += leak_leak[meter]
    
    return np.sqrt(loss_total / length)

In [14]:
np.random.seed(1)

score = [list(), list(), list(), list()]
weight = [list(), list(), list(), list()]

for meter in [0,1,2,3]:
    f = partial(optimization, meter, sub_sub, leak_sub, leak_leak, len(focus_df.query(f'meter=={meter}')))
    for i in tqdm(range(100000)):
        W = np.random.rand(N)

        to_zero = np.arange(N)
        np.random.shuffle(to_zero)

        W[to_zero[:np.random.randint(N)]] = 0
        W /= W.sum()
        W *= np.random.rand() * 0.3 + 0.8
        score[meter].append(f(W))
        weight[meter].append(W)
    
    score[meter] = np.array(score[meter])
    weight[meter] = np.array(weight[meter])


100%|██████████| 100000/100000 [00:06<00:00, 15444.24it/s]


In [15]:
# 今までのbest_score
# 0
# 0.6457341965029569 # 2,4,15
# 0.7670864405850898 # 1,2,15

# 1
# 1.1028821246797627

# 2
# 0.9266132278805228

# 3
# 1.1595900581550331


# new leak set
# 0.7229769339789924
# 1.4360871220006104
# 0.9269688704973097
# 1.1567657302973289

# 0.723733378801817
# 1.4325472242980108
# 0.9270139246582472
# 1.1560000215450374

# 0.7232916263657445
# 1.4257850382841468
# 0.926444086189883
# 1.1553662804048155

In [16]:
for meter in [0,1,2,3]:
    print(score[meter].min())
    print(weight[meter][score[meter].argmin()].sum())
    print()

0.42098025406751793
1.0022769123840924

1.1634973907282613
1.003084790662937

0.8652917291366509
1.0101346504900341

1.0325176442373727
0.9977065743799649



In [17]:
for meter in [0,1,2,3]:
#     for i in range(N):
    print(weight[meter][score[meter].argmin()])
    print()

[0.0166223  0.         0.         0.2458813  0.         0.73977331]

[0.25009516 0.         0.18585956 0.         0.         0.56713007]

[0.         0.         0.         0.19816822 0.         0.81196643]

[0.2170427  0.         0.19712148 0.         0.58354239 0.        ]



In [18]:
def new_pred(test_sub, weight, N):
    pred_new = list()
    for meter in [0,1,2,3]:
        test_sub_m = test_sub.query(f'meter=={meter}')
        ensemble_m = sum([np.log1p(test_sub_m[f'pred{i+1}']) * weight[meter][score[meter].argmin()][i] for i in range(N)])
        pred_new.append(ensemble_m)

    pred_new = pd.concat(pred_new).sort_index()
    return np.expm1(pred_new)

In [19]:
ensembled_pred1 = new_pred(test_sub, weight, N)

In [20]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test.index]))))

0.7855268527671803
0.7664519155946278
0.7759848514773457


In [19]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test.index]))))

0.7919666357312548
0.7757227819125097
0.7838330149095364


In [21]:
# 0.9455117442226527
# 0.7985375584144594

# 0.9404366275680945
# 0.8062714386695903

#0.9408301210841344
#0.8029139931610979
#all 0.8744504105324552

# 0.9382531143644084
# 0.8031481393084626
# 0.873175711300433

In [22]:
new_submission = pd.read_csv('../input/sample_submission.csv')

In [23]:
new_submission['meter_reading'] = ensembled_pred1.values
#                                       submission1['meter_reading'] * arr[arr[:,0].argmin()][1][0] \
#                                     + submission2['meter_reading'] * arr[arr[:,0].argmin()][1][1] \
#                                     + submission3['meter_reading'] * arr[arr[:,0].argmin()][1][2]

In [24]:
new_submission.to_csv('../output/submission_my_leak_validation.csv', index=False)

In [ ]:
new_submission